In [1]:
import pickle

import pandas as pd
import numpy as np

from selenium import webdriver

from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions

import time
import random
import os

In [2]:
def set_ff_preferences():
    profile = webdriver.FirefoxProfile()
    profile.set_preference('browser.download.folderList', 2) # custom location
    profile.set_preference('browser.download.dir', (os.getcwd()+'/data/Cash_Flows'))
    profile.set_preference('browser.download.manager.showWhenStarting', False)
    profile.set_preference('browser.helperApps.neverAsk.saveToDisk', "application/csv, text/csv")
    return profile

def create_empty_ff_driver():
    profile = set_ff_preferences()
    a_driver = webdriver.Firefox(firefox_profile=profile)
    return a_driver

def create_driver_opening_url():
    a_driver = create_empty_ff_driver()
    return a_driver

def clean_from_text(text):
#     cleaned_text = text.replace(",","").strip()
    cleaned_text = text.strip()
    return cleaned_text

In [3]:
def get_flatdata():
    urls = {'https://dividenddetective.com/big_dividend_list.htm': '/html/body/div[2]/div/div[1]/div[2]/div[9]', 
        'https://dividenddetective.com/big_dividend_list2.htm': '/html/body/div[2]/div/div[1]/div/div[9]', 
        'https://dividenddetective.com/big_dividend_list3.htm': '/html/body/div[2]/div/div[1]/div/div[6]', 
        'https://dividenddetective.com/big_dividend_list4.htm': '/html/body/div[2]/div/div[1]/div/div[8]'
       }

    flatdata = []

    a_driver = create_driver_opening_url()
    
    
    for key in urls:
    
        a_driver.get(key)

        tableDiv = WebDriverWait(a_driver, 10).until(
            expected_conditions.visibility_of_element_located(
            (By.XPATH, urls[key])))

        page_data = tableDiv.text.split('\n')
        flatdata = flatdata + page_data
        
    a_driver.close()

    return flatdata

In [4]:
def format_data(flat_data, columns=4):
    num_dividend_stocks = int(len(flat_data)/columns)
    square_data = np.array(flat_data).reshape(num_dividend_stocks,4)
    df = pd.DataFrame(square_data)
    df.columns = ['ticker','name','annual_div','div_yield']
    df['annual_div'] = df['annual_div'].astype(float)
    df['div_yield'] = df['div_yield'].astype(float)
    return df

In [5]:
def get_data():
    return format_data(get_flatdata())

,ticker,name,annual_div,div_yield
0,FCCY,1st Constitution Bancorp,0.36,2.8
1,SRCE,1st Source,1.12,3.4
2,MMM,3M Company,5.88,4.0
3,ABBV,AbbVie,4.72,5.6
4,ACCO,ACCO Brands,0.26,4.1
...,...,...,...,...
740,WYND,Wyndham Destinations,1.85,7.3
741,WH,Wyndham Hotels & Resorts,1.28,2.9
742,XEL,Xcel Energy,1.72,2.9
743,XRX,Xerox Holdings,1.00,5.5
